#### Name: Mahmoud Hamdy Mahmoud
#### Sec: 2    
#### BN: 24

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 

In [2]:
rating_data = pd.read_csv('ratings.csv')
rating_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies_data = pd.read_csv('movies.csv')
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies_data[movies_data['movieId']<= 233].count()

movieId    200
title      200
genres     200
dtype: int64

In [7]:
#Selecting 200 movies which are corresponding to the first 200 users
data = rating_data.copy()
data = data[data['userId']<=200]
data = data[data['movieId'].isin(movies_data[:200]['movieId'])]
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Recommendation class

In [165]:
from sklearn.metrics.pairwise import cosine_similarity
class RecomindMovie:
    def __init__(self, rating_data, movies_data, data):
        self.rating_data = rating_data
        self.movies_data = movies_data
        self.movies = data
        self.merge = pd.merge(self.rating_data,self.movies_data,on='movieId')
        # self.mat = pd.DataFrame()
    def getSimilaritymat(self, data_mat):
        #Choose to fill the missing values with 0
        data_mat = data_mat.fillna(0, axis = 1)
        #Or we can Choose to fill the missing values with the mean of the row or the column
        # data_mat = data_mat.apply(lambda col: col.fillna(col.mean()), axis=0)
        #call the cosine similarity function
        mat = cosine_similarity(data_mat)
        #Return the similarity matrix
        mat = pd.DataFrame(mat,index=data_mat.index,columns=data_mat.index)
        return mat
    
    def get_movieBased_similarity(self):
        #Get the pivot table of the movies aka the features
        movieRating_feature= self.movies.pivot_table(index='movieId',columns='userId',values='rating')
        #call the function to get the similarity matrix
        simMat = self.getSimilaritymat(movieRating_feature)
        #Return the similarity matrix
        return pd.DataFrame(simMat,index=movieRating_feature.index,columns=movieRating_feature.index)
    
    def get_userBased_similarity(self):
        #To get the similarity between the users, we need to transpose the matrix or switch the indices manually
        userRating_feature= self.movies.pivot_table(index='userId',columns='movieId',values='rating')
        #call the function to get the similarity matrix
        simMat = self.getSimilaritymat(userRating_feature)
        #Return the similarity matrix
        return pd.DataFrame(simMat,index=userRating_feature.index,columns=userRating_feature.index)
    
    def recommendMovies(self, movie_name, num=10):
        #Get the id of the movie
        id = self.movies_data[self.movies_data['title']==movie_name]['movieId'].values[0]
        #Sort the similarity matrix based on the movie id then get the top n movies and store their id's
        ids = self.get_movieBased_similarity()[id].sort_values(ascending=False)[1:num+1].index
        #Return the titles of the movies
        return self.movies_data[self.movies_data['movieId'].isin(ids)]['title']
    
    def recommendMoviesBasedOnUser(self, user_id, number=3, nieghbours=10, decreaseNeghbour=True):
        self.merge.fillna(0)
        #Get the top n neighbours of the user
        ids = self.get_userBased_similarity()[user_id].sort_values(ascending=False)[1:nieghbours+1].index
        #Iterate on them to get the top movies based on their ratings
        movie_id = []
        for num in ids:
            #Get the top rated movie of the user
            movie = self.rating_data[self.rating_data['userId']==num]['rating'].sort_values(ascending=False).index
            if(decreaseNeghbour):
                #Get the heighst movie accorging to the user's rating
                movie_id.append(rating_data['movieId'].loc[movie[0]])
            else:
            #We can select the top n movies based on the user's rating
                movie_id.append(rating_data['movieId'].loc[movie[:number]])
        movie_id = np.array(movie_id).flatten()
        # print(movie_id)
        counter = 0
        titles = []
        #Iterate on the movies to get the titles
        for m in movie_id:
            #Check whether the array is empty or not and the number of movies is less than the number of movies to be recommended
            if((rating_data[rating_data['movieId'] == m].loc[rating_data['userId'] == 200].values).size==0) \
            & (counter<number):
                #If the movie is not rated by the user then the array is empty, then append it to the list
                titles.append(self.movies_data[self.movies_data['movieId'] == m]['title'].values[0])
                print(self.movies_data[self.movies_data['movieId'] == m]['title'].values[0])
                # print("The id counter ",number)
                counter+=1
        # counter = 0
        return titles

In [166]:
data = rating_data.copy()
data = data[data['userId']<=200]
data = data[data['movieId'].isin(movies_data[:200]['movieId'])]
movies = RecomindMovie(rating_data,movies_data, data)


### 1. Similarity Matrix in MovieBasedRecommendation

In [168]:
SimMat_MovieBased = pd.DataFrame(movies.get_movieBased_similarity())
SimMat_MovieBased

movieId,1,2,3,4,5,6,7,8,9,10,...,223,224,225,227,228,229,230,231,232,233
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.364634,0.284825,0.000000,0.279775,0.327928,0.270701,0.128765,0.195017,0.352899,...,0.363093,0.211791,0.160623,0.088024,0.107733,0.104516,0.140704,0.360625,0.087884,0.000000
2,0.364634,1.000000,0.278446,0.102937,0.277072,0.278936,0.240488,0.130496,0.000000,0.342590,...,0.202565,0.085855,0.132766,0.114888,0.000000,0.000000,0.191316,0.352531,0.000000,0.000000
3,0.284825,0.278446,1.000000,0.150278,0.391915,0.341700,0.592157,0.406427,0.288534,0.231410,...,0.284289,0.200545,0.347200,0.197325,0.000000,0.000000,0.244260,0.485324,0.000000,0.000000
4,0.000000,0.102937,0.150278,1.000000,0.239259,0.176604,0.297614,0.253546,0.000000,0.129040,...,0.108721,0.382276,0.280386,0.000000,0.000000,0.000000,0.369406,0.132357,0.257513,0.000000
5,0.279775,0.277072,0.391915,0.239259,1.000000,0.256166,0.539392,0.404422,0.000000,0.275293,...,0.150873,0.295177,0.380707,0.070687,0.000000,0.000000,0.287247,0.296883,0.092419,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,0.104516,0.000000,0.000000,0.000000,0.000000,0.242299,0.091873,0.000000,0.000000,0.000000,...,0.253580,0.238400,0.000000,0.000000,0.000000,1.000000,0.316763,0.000000,0.000000,0.000000
230,0.140704,0.191316,0.244260,0.369406,0.287247,0.305806,0.346312,0.234154,0.000000,0.154921,...,0.325314,0.481415,0.258941,0.109137,0.326512,0.316763,1.000000,0.142605,0.256842,0.142543
231,0.360625,0.352531,0.485324,0.132357,0.296883,0.295106,0.357803,0.178979,0.079414,0.411800,...,0.335287,0.142591,0.487390,0.301241,0.187180,0.000000,0.142605,1.000000,0.000000,0.068097


### 2. a) Toy Story Recommendations

In [169]:
movies.recommendMovies('Toy Story (1995)')

32                           Babe (1995)
43           Seven (a.k.a. Se7en) (1995)
46            Usual Suspects, The (1995)
92                  Happy Gilmore (1996)
97                     Braveheart (1995)
98                    Taxi Driver (1976)
123                     Apollo 13 (1995)
126                Batman Forever (1995)
138    Die Hard: With a Vengeance (1995)
176                    Waterworld (1995)
Name: title, dtype: object

### 2. b) Waiting to Exhale recommendations

In [170]:
movies.recommendMovies('Waiting to Exhale (1995)')

50                                     Georgia (1995)
100                           Before and After (1996)
151                                   Mad Love (1995)
153    Mighty Morphin Power Rangers: The Movie (1995)
160                                   Reckless (1995)
169                               Three Wishes (1995)
177                         White Man's Burden (1995)
180                                Bushwhacked (1995)
182             Before the Rain (Pred dozhdot) (1994)
187                                  Cure, The (1995)
Name: title, dtype: object

### 3. Recommendation based on user (Bonus)

In [171]:
titles = movies.recommendMoviesBasedOnUser(200, 3, 10)#By selcting to choose the top rated movie of each neighbour user

Spotlight (2015)
Motorcycle Diaries, The (Diarios de motocicleta) (2004)
Jungle Book, The (1967)


In [173]:
titles1 = movies.recommendMoviesBasedOnUser(200, 3, 10, False )#By selcting to choose the top 3 rated movie of each neighbour user

Up (2009)
Terminator 2: Judgment Day (1991)
Star Trek: Generations (1994)


### UserBased Similarity Matrix

In [103]:
SimMat_UserBased = pd.DataFrame(movies.get_userBased_similarity())
SimMat_UserBased.head()

userId,1,3,4,5,6,7,8,9,11,12,...,191,192,193,194,195,196,197,198,199,200
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.042108,0.241800,0.254840,0.260443,0.295979,0.122551,0.190710,0.000000,...,0.281312,0.143808,0.238197,0.177761,0.230797,0.099823,0.247739,0.216804,0.257020,0.224353
3,0.000000,1.000000,0.000000,0.000000,0.085023,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.042108,0.000000,1.000000,0.259514,0.094189,0.068724,0.118673,0.000000,0.000000,0.043519,...,0.274774,0.037947,0.000000,0.000000,0.051709,0.000000,0.000000,0.249136,0.091558,0.066601
5,0.241800,0.000000,0.259514,1.000000,0.196962,0.543902,0.478846,0.000000,0.375117,0.115342,...,0.392683,0.188576,0.312348,0.000000,0.274093,0.271146,0.389833,0.426444,0.337031,0.390373
6,0.254840,0.085023,0.094189,0.196962,1.000000,0.105589,0.358563,0.058325,0.329019,0.156984,...,0.299079,0.286443,0.000000,0.149462,0.271497,0.235846,0.070743,0.299563,0.272168,0.301846
